In [1]:
import numpy as np
import pandas as pd

from datetime import datetime
from IPython.display import display
import cv2
import os

import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Concatenate, GlobalMaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Permute, multiply
#from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

from tensorflow.keras.applications.xception import Xception, preprocess_input

from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard, LambdaCallback
from tensorflow.keras.utils import to_categorical


import imgaug as ia
from imgaug import augmenters as iaa

In [2]:
def create_augmenter(train=True):
    # from https://github.com/aleju/imgaug
    # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
    # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image
    # All augmenters with per_channel=0.5 will sample one value _per image_
    # in 50% of all cases. In all other cases they will sample new values
    # _per channel_.
    if train:
        seq = iaa.Sequential(
            [
                # apply the following augmenters to most images
                iaa.Fliplr(0.5), # horizontally flip 50% of all images
                iaa.Flipud(0.2), # vertically flip 20% of all images
                # crop images by -5% to 10% of their height/width
                sometimes(iaa.CropAndPad(
                    percent=(-0.05, 0.1),
                    pad_mode=ia.ALL, # random mode from all available modes will be sampled per image.
                    pad_cval=(0, 255) # The constant value to use if the pad mode is constant or the end value to use if the mode is linear_ramp
                )),
                sometimes(iaa.Affine(
                    scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
                    rotate=(-45, 45), # rotate by -45 to +45 degrees
                    shear=(-16, 16), # shear by -16 to +16 degrees
                    cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                    mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                )),
            ],
        )
    else:
        pass
    return seq

In [79]:
# reference: https://www.kaggle.com/mpalermo/keras-pipeline-custom-generator-imgaug
class BaseDataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, images_paths, labels, batch_size=64, image_dimensions = (512, 512, 3),
                 shuffle=False, augmenter=None, preprocessor=None,
                 return_label=True, total_classes=None):
        self.labels       = labels              # array of labels
        self.images_paths = images_paths        # array of image paths
        self.dim          = image_dimensions    # image dimensions
        self.batch_size   = batch_size          # batch size
        self.shuffle      = shuffle             # shuffle bool
        self.augmenter      = augmenter           # augmenter
        self.preprocessor = preprocessor
        self.return_label = return_label
        self.total_classes = total_classes
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images_paths) / self.batch_size))

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.images_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def gather_batch_item(self, index):
        'Generate one batch of data'
        # selects indices of data for next batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # select data and load images
        images = [cv2.imread(self.images_paths[k]) for k in indexes]

        # preprocess and augment data
        if self.augmenter:
            images = self.augmenter.augment_images(images)

        images= np.array([self.preprocess_image(cv2.resize(img, self.dim[:2])) for img in images])
        
        if self.return_label:
            labels = np.array([self.labels[k] for k in indexes])
            labels = to_categorical(labels, num_classes=self.total_classes)
            return images, labels
        else:
            return images
    
    def __getitem__(self, index):
        return self.gather_batch_item(index)
        
    def preprocess_image(self, images):
        if self.preprocessor is None:
            images = images / 255.
            pass
        else:
            images = self.preprocessor(images)
        return images
    
class MultiOutputDataGenerator(BaseDataGenerator):
    'Generates multiple output data for Keras'
    def __init__(self, images_paths, labels, batch_size=64, image_dimensions = (512, 512, 3),
                 shuffle=False, augmenter=None, preprocessor=None,
                 return_label=True, total_classes=None, output_names=None, tta_augmentors=None):
        # Init parent's parameter
        super().__init__(images_paths,
                labels, batch_size, image_dimensions,
                 shuffle, augmenter, preprocessor,
                 return_label, total_classes)
        
        self.output_names = output_names
        self.tta_augmentors = tta_augmentors
    
    def __getitem__(self, index):
        if self.return_label:
            images, labels = self.gather_batch_item(index)
            output_dict = {}
            # Copy labels to each output name
            for output_name in self.output_names:
                output_dict[output_name] = labels
            if self.tta_augmentors != None:
                images = self.get_tta_images(images)
            return images, output_dict
        else:
            images = self.gather_batch_item(index)
            if self.tta_augmentors != None:
                images = self.get_tta_images(images)
            return images
    def get_tta_images(self, images):
        aug_images = []
        # Original
        aug_images.append(images)
        for augmentor in self.tta_augmentors:
            aug_images.append(augmentor.augment_images(images))
        images = aug_images
        return images

In [4]:
# Xception pretrain model input size
image_shape = (299, 299, 3)

# 133
total_classes = len(os.listdir('dogImages/train')) 
batch_size = 20

In [5]:
def create_path_labels(path):
    all_classes = os.listdir(path)
    label_dict = {}
    img_paths, label_list = [], []
    for label_name in all_classes:
        label_num, dog_name = label_name.split('.')
        # Start with 0
        label_num = int(label_num) - 1
        label_dict[int(label_num)] = dog_name
        for image_name in os.listdir(path + '/' + label_name):
            img_paths.append(path + '/' + label_name + '/' + image_name)
            label_list.append(label_num)
    df = pd.DataFrame({'img_path': img_paths, 'label': label_list})
    return label_dict, df

In [6]:
num_label_dict, df_train = create_path_labels('dogImages/train')
_, df_val = create_path_labels('dogImages/valid')
_, df_test = create_path_labels('dogImages/test')

In [7]:
num_label_dict

{0: 'Affenpinscher',
 1: 'Afghan_hound',
 2: 'Airedale_terrier',
 3: 'Akita',
 4: 'Alaskan_malamute',
 5: 'American_eskimo_dog',
 6: 'American_foxhound',
 7: 'American_staffordshire_terrier',
 8: 'American_water_spaniel',
 9: 'Anatolian_shepherd_dog',
 10: 'Australian_cattle_dog',
 11: 'Australian_shepherd',
 12: 'Australian_terrier',
 13: 'Basenji',
 14: 'Basset_hound',
 15: 'Beagle',
 16: 'Bearded_collie',
 17: 'Beauceron',
 18: 'Bedlington_terrier',
 19: 'Belgian_malinois',
 20: 'Belgian_sheepdog',
 21: 'Belgian_tervuren',
 22: 'Bernese_mountain_dog',
 23: 'Bichon_frise',
 24: 'Black_and_tan_coonhound',
 25: 'Black_russian_terrier',
 26: 'Bloodhound',
 27: 'Bluetick_coonhound',
 28: 'Border_collie',
 29: 'Border_terrier',
 30: 'Borzoi',
 31: 'Boston_terrier',
 32: 'Bouvier_des_flandres',
 33: 'Boxer',
 34: 'Boykin_spaniel',
 35: 'Briard',
 36: 'Brittany',
 37: 'Brussels_griffon',
 38: 'Bull_terrier',
 39: 'Bulldog',
 40: 'Bullmastiff',
 41: 'Cairn_terrier',
 42: 'Canaan_dog',
 43: '

In [57]:
display(df_train.head())
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

,img_path,label
0,dogImages/train/001.Affenpinscher/Affenpinsche...,0
1,dogImages/train/001.Affenpinscher/Affenpinsche...,0
2,dogImages/train/001.Affenpinscher/Affenpinsche...,0
3,dogImages/train/001.Affenpinscher/Affenpinsche...,0
4,dogImages/train/001.Affenpinscher/Affenpinsche...,0


(6680, 2)
(835, 2)
(836, 2)


In [104]:
train_datagen = MultiOutputDataGenerator(images_paths=df_train['img_path'].values, labels=df_train['label'].values,
                              batch_size=batch_size, image_dimensions=image_shape, shuffle=True,
                              augmenter=create_augmenter(train=True), preprocessor=preprocess_input,
                             return_label=True, total_classes=total_classes, output_names=['original_out', 'se_out'])

val_datagen = MultiOutputDataGenerator(images_paths=df_val['img_path'].values, labels=df_val['label'].values,
                              batch_size=5, image_dimensions=image_shape, shuffle=True,
                              augmenter=None,
                                preprocessor=preprocess_input,
                                return_label=True, total_classes=total_classes, output_names=['original_out', 'se_out'])

test_datagen = MultiOutputDataGenerator(images_paths=df_test['img_path'].values, labels=df_test['label'].values,
                              batch_size=1, image_dimensions=image_shape, shuffle=False,
                              augmenter=None,
                                preprocessor=preprocess_input,
                            return_label=False, total_classes=total_classes, output_names=['original_out', 'se_out'])

In [128]:
print(len(train_datagen))
print(len(val_datagen))
print(len(test_datagen))

334
167
836


In [10]:
def squeeze_excite_block(tensor, ratio=16):
    # From: https://github.com/titu1994/keras-squeeze-excite-network
    init = tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = K.int_shape(init)[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x

In [11]:
# pooling = None - output shape (None, 8, 8, 1536)
# pooling = max  - output shape (None, 1536)
# pooling = avg  - output shape (None, 1536)

xception = Xception(include_top=False, weights='imagenet', input_shape=image_shape, pooling=None)
x = xception.output

# Original branch
gavg = GlobalAveragePooling2D()(x)
gmax = GlobalMaxPooling2D()(x)
original_concat = Concatenate(axis=-1)([gavg, gmax,])
original_concat = Dropout(0.5)(original_concat)
original_final = Dense(total_classes, activation='softmax', name='original_out')(original_concat)

# SE branch
se_out = squeeze_excite_block(x)
se_gavg = GlobalAveragePooling2D()(se_out)
se_gmax = GlobalMaxPooling2D()(se_out)
se_concat = Concatenate(axis=-1)([se_gavg, se_gmax,])
se_concat = Dropout(0.5)(se_concat)
se_final = Dense(total_classes, activation='softmax', name='se_out')(se_concat)

model = Model(inputs=xception.input, outputs=[original_final, se_final])

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.Xception_best.hdf5', 
                                  verbose=1, save_best_only=True)


logdir = f".\logs\warmup"
# Create target Directory if don't exist
if not os.path.exists(logdir):
    os.mkdir(logdir)
    

tensorboard_callback = TensorBoard(log_dir=logdir)

early_stop = EarlyStopping(monitor="val_loss",
                               mode="min",
                               patience=15,
                               restore_best_weights=True)

# Maybe useful, haven't tried
def scheduler(epoch):
    if epoch < 50:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 * (10 - epoch))

lr_scheduler = LearningRateScheduler(scheduler)

## Warm up

In [14]:
# Freeze pretrained part
for layer in model.layers[:-15]:
    layer.trainable = False
    
model.compile(optimizer='adam',
              loss={'original_out': 'categorical_crossentropy', 'se_out': 'categorical_crossentropy'},
              loss_weights={'original_out': 1., 'se_out': 1.}, metrics=['accuracy'])

In [15]:
history = model.fit_generator(generator=train_datagen,
                                       validation_data=val_datagen,
                                       epochs=5,
                                       callbacks=[tensorboard_callback, early_stop, checkpointer],
                                       verbose=1,
                                       )

Epoch 1/5
333/334 [============================>.] - ETA: 0s - loss: 7.0953 - original_out_loss: 4.2644 - se_out_loss: 2.8309 - original_out_accuracy: 0.3278 - se_out_accuracy: 0.3772
Epoch 00001: val_loss improved from inf to 1.92039, saving model to saved_models/weights.best.Xception.hdf5
334/334 [==============================] - 279s 835ms/step - loss: 7.0894 - original_out_loss: 4.2609 - se_out_loss: 2.8286 - original_out_accuracy: 0.3281 - se_out_accuracy: 0.3774 - val_loss: 1.9204 - val_original_out_loss: 1.0282 - val_se_out_loss: 0.8921 - val_original_out_accuracy: 0.7246 - val_se_out_accuracy: 0.7377
Epoch 2/5
333/334 [============================>.] - ETA: 0s - loss: 4.8835 - original_out_loss: 3.1104 - se_out_loss: 1.7730 - original_out_accuracy: 0.4893 - se_out_accuracy: 0.5506
Epoch 00002: val_loss improved from 1.92039 to 1.58128, saving model to saved_models/weights.best.Xception.hdf5
334/334 [==============================] - 175s 524ms/step - loss: 4.8830 - original_ou

## Train whole model

In [16]:
# Freeze pretrained part
for layer in model.layers[:-15]:
    layer.trainable = True

logdir = f".\logs\whole"
# Create target Directory if don't exist
if not os.path.exists(logdir):
    os.mkdir(logdir)

tensorboard_callback = TensorBoard(log_dir=logdir)


model.compile(optimizer='sgd',
              loss={'original_out': 'categorical_crossentropy', 'se_out': 'categorical_crossentropy'},
              loss_weights={'original_out': 1., 'se_out': 1.}, metrics=['accuracy'])

In [17]:
history = model.fit_generator(generator=train_datagen,
                                       validation_data=val_datagen,
                                       epochs=100,
                                       callbacks=[tensorboard_callback, checkpointer],
                                       verbose=1,
                                       )

Epoch 1/100
333/334 [============================>.] - ETA: 0s - loss: 2.1426 - original_out_loss: 1.1894 - se_out_loss: 0.9533 - original_out_accuracy: 0.6983 - se_out_accuracy: 0.7351
Epoch 00001: val_loss improved from 1.44479 to 1.01614, saving model to saved_models/weights.best.Xception.hdf5
334/334 [==============================] - 207s 620ms/step - loss: 2.1399 - original_out_loss: 1.1879 - se_out_loss: 0.9520 - original_out_accuracy: 0.6987 - se_out_accuracy: 0.7353 - val_loss: 1.0161 - val_original_out_loss: 0.5506 - val_se_out_loss: 0.4656 - val_original_out_accuracy: 0.8359 - val_se_out_accuracy: 0.8527
Epoch 2/100
333/334 [============================>.] - ETA: 0s - loss: 1.3541 - original_out_loss: 0.7303 - se_out_loss: 0.6238 - original_out_accuracy: 0.7959 - se_out_accuracy: 0.8141
Epoch 00002: val_loss improved from 1.01614 to 0.84061, saving model to saved_models/weights.best.Xception.hdf5
334/334 [==============================] - 208s 624ms/step - loss: 1.3536 - ori

Epoch 30/100
333/334 [============================>.] - ETA: 0s - loss: 0.1301 - original_out_loss: 0.0700 - se_out_loss: 0.0601 - original_out_accuracy: 0.9788 - se_out_accuracy: 0.9832
Epoch 00030: val_loss did not improve from 0.81592
334/334 [==============================] - 230s 689ms/step - loss: 0.1298 - original_out_loss: 0.0698 - se_out_loss: 0.0600 - original_out_accuracy: 0.9789 - se_out_accuracy: 0.9832 - val_loss: 0.9377 - val_original_out_loss: 0.5699 - val_se_out_loss: 0.3677 - val_original_out_accuracy: 0.8910 - val_se_out_accuracy: 0.8934
Epoch 31/100
333/334 [============================>.] - ETA: 0s - loss: 0.1189 - original_out_loss: 0.0590 - se_out_loss: 0.0600 - original_out_accuracy: 0.9824 - se_out_accuracy: 0.9824
Epoch 00031: val_loss did not improve from 0.81592
334/334 [==============================] - 227s 680ms/step - loss: 0.1186 - original_out_loss: 0.0588 - se_out_loss: 0.0598 - original_out_accuracy: 0.9825 - se_out_accuracy: 0.9825 - val_loss: 0.962

KeyboardInterrupt: 

In [18]:
model.save_weights('saved_models/weights.best.Xception_whole_model.hdf5')

In [19]:
K.clear_session()

In [102]:
model.load_weights('saved_models/weights.best.Xception_whole_model.hdf5')

In [105]:
pred = model.predict_generator(generator=test_datagen,
                                       verbose=1,
                                       )

836/836 [==============================] - 45s 54ms/step


In [106]:
def combine_prediction(predictions, weights=[1.,1.]):
    predictions = np.array(predictions)
    weights = np.array(weights).reshape(predictions.shape[0], 1, 1)
    return np.mean(np.multiply(pred, weights), axis=0)

def cal_accuracy(predictions, truth):
    if type(predictions) != list:
        predictions = [predictions]
    accuracy = []
    for prediction in predictions:
        prediction = np.argmax(prediction, axis=-1)
        correct_nums = (prediction == truth).sum()
        accuracy.append(correct_nums / len(prediction))
    return accuracy

In [107]:
# Same weight
cal_accuracy(combine_prediction(pred, [1., 1.]), df_test['label'].values)

[0.8971291866028708]

## Load best model

In [24]:
model.load_weights('saved_models/weights.best.Xception_best.hdf5')

In [25]:
pred = model.predict_generator(generator=test_datagen,
                                       verbose=1,
                                       )

836/836 [==============================] - 48s 58ms/step


In [71]:
len(df_test['label'].values)

836

In [26]:
# Same weight
cal_accuracy(combine_prediction(pred, [1., 1.]), df_test['label'].values)

[0.8911483253588517]

## Test Time Augmentation

In [27]:
# Get Augmentors
tta_augmentors = [iaa.Fliplr(1.), iaa.Flipud(1.)]


In [36]:
tta_test_datagen = MultiOutputDataGenerator(images_paths=df_test['img_path'].values, labels=df_test['label'].values,
                              batch_size=1, image_dimensions=image_shape, shuffle=False,
                              augmenter=None,
                                preprocessor=preprocess_input,
                            return_label=False, total_classes=total_classes, output_names=['original_out', 'se_out'], tta_augmentors=tta_augmentors)

In [37]:
total_classes

133

In [41]:
all_predictions = np.zeros((df_test.shape[0], total_classes))
count = 0
total_len = len(tta_test_datagen)
for images in tta_test_datagen:
    count += 1
    print("{}/{}".format(count, total_len), end="\r")
    preds = []
    for image in images:
        pred = model.predict_on_batch(image)
        pred = combine_prediction(pred, [1.0, 1.0])
        preds.append(pred)
    all_predictions[count-1] = combine_prediction(preds, [1.0, 1.0, 1.0])

In [42]:
cal_accuracy(np.array(all_predictions).reshape((836, 133)), df_test['label'].values)

[0.7894736842105263]

## Pseudo labeling

In [60]:
val_pred = model.predict_generator(generator=val_datagen,
                                       verbose=1,
                                       )
val_pred = combine_prediction(val_pred, [1., 1.])

167/167 [==============================] - 10s 62ms/step


In [97]:
test_datagen = MultiOutputDataGenerator(images_paths=df_test['img_path'].values, labels=df_test['label'].values,
                              batch_size=2, image_dimensions=image_shape, shuffle=False,
                              augmenter=None,
                                preprocessor=preprocess_input,
                            return_label=False, total_classes=total_classes, output_names=['original_out', 'se_out'])

test_pred = model.predict_generator(generator=test_datagen,
                                       verbose=1,
                                       )

test_pred = combine_prediction(test_pred, [1., 1.])

418/418 [==============================] - 24s 57ms/step


In [98]:
val_pred.shape

(835, 133)

In [99]:
test_pred.shape

(836, 133)

In [100]:
val_pseudolabels= np.argmax(val_pred, axis=-1)
test_pseudolabels = np.argmax(test_pred, axis=-1)

In [108]:
print(val_pseudolabels.shape)

print(test_pseudolabels.shape)

(835,)
(836,)


### Combine pseudo labels with train data

In [112]:
pseudo_data = pd.DataFrame({"img_path": np.concatenate([df_val['img_path'].values, df_test['img_path'].values]),
                            "label": np.concatenate([val_pseudolabels, test_pseudolabels])})

In [113]:
pseudo_train = pd.concat([pseudo_data, df_train], axis=0)

In [114]:
pseudo_train.shape

(8351, 2)

In [115]:
pseudo_train_datagen = MultiOutputDataGenerator(images_paths=pseudo_train['img_path'].values, labels=pseudo_train['label'].values,
                              batch_size=batch_size, image_dimensions=image_shape, shuffle=True,
                              augmenter=create_augmenter(train=True), preprocessor=preprocess_input,
                             return_label=True, total_classes=total_classes, output_names=['original_out', 'se_out'])

In [118]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.Xception_PL.hdf5', 
                                  verbose=1, save_best_only=True)

logdir = f".\logs\pseudolabeling"
# Create target Directory if don't exist
if not os.path.exists(logdir):
    os.mkdir(logdir)

tensorboard_callback = TensorBoard(log_dir=logdir)

early_stop = EarlyStopping(monitor="val_loss",
                               mode="min",
                               patience=15,
                               restore_best_weights=True)

In [119]:
model.load_weights('saved_models/weights.best.Xception_whole_model.hdf5')

In [120]:
# Freeze pretrained part
for layer in model.layers[:-15]:
    layer.trainable = True
    
sgd = SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=True)

model.compile(optimizer=sgd,
              loss={'original_out': 'categorical_crossentropy', 'se_out': 'categorical_crossentropy'},
              loss_weights={'original_out': 1., 'se_out': 1.}, metrics=['accuracy'])

In [121]:
history = model.fit_generator(generator=pseudo_train_datagen,
                                       validation_data=val_datagen,
                                       epochs=100,
                                       callbacks=[tensorboard_callback, early_stop, checkpointer, lr_scheduler],
                                       verbose=1,
                                       )

Epoch 1/100
416/417 [============================>.] - ETA: 0s - loss: 2.7612 - original_out_loss: 1.4737 - se_out_loss: 1.2875 - original_out_accuracy: 0.8739 - se_out_accuracy: 0.8787
Epoch 00001: val_loss improved from inf to 0.78541, saving model to saved_models/weights.best.Xception_PL.hdf5
417/417 [==============================] - 279s 668ms/step - loss: 2.7618 - original_out_loss: 1.4742 - se_out_loss: 1.2876 - original_out_accuracy: 0.8737 - se_out_accuracy: 0.8784 - val_loss: 0.7854 - val_original_out_loss: 0.3316 - val_se_out_loss: 0.4538 - val_original_out_accuracy: 0.9018 - val_se_out_accuracy: 0.9102
Epoch 2/100
416/417 [============================>.] - ETA: 0s - loss: 2.3207 - original_out_loss: 1.1954 - se_out_loss: 1.1253 - original_out_accuracy: 0.8754 - se_out_accuracy: 0.8782
Epoch 00002: val_loss did not improve from 0.78541
417/417 [==============================] - 279s 668ms/step - loss: 2.3185 - original_out_loss: 1.1938 - se_out_loss: 1.1247 - original_out_ac

Epoch 16/100
131/417 [========>.....................] - ETA: 3:08 - loss: 1.7145 - original_out_loss: 0.8450 - se_out_loss: 0.8695 - original_out_accuracy: 0.8893 - se_out_accuracy: 0.8916

KeyboardInterrupt: 

In [123]:
pred = model.predict_generator(generator=test_datagen,
                                       verbose=1,
                                       )

836/836 [==============================] - 48s 57ms/step


In [124]:
# Same weight
cal_accuracy(combine_prediction(pred, [1., 1.]), df_test['label'].values)

[0.895933014354067]

## Load best model

In [126]:
model.load_weights('saved_models/weights.best.Xception_PL.hdf5')

In [127]:
pred = model.predict_generator(generator=test_datagen,
                                       verbose=1,
                                       )
# Same weight
cal_accuracy(combine_prediction(pred, [1., 1.]), df_test['label'].values)

836/836 [==============================] - 48s 57ms/step


[0.90311004784689]